In [ ]:
"""
Professional Weather Application with Gradio Interface
Designed for Google Colab - Copy and run this entire code in a Colab notebook
Enhanced with light, smooth colors and additional features
"""

# Install required packages
!pip install -q gradio requests plotly pandas

import gradio as gr
import requests
import json
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from typing import Dict, List, Tuple
import pandas as pd

# Your Weather API Key
API_KEY = "9be813d77c5496f42683c9f2966a9e89"
BASE_URL = "http://api.openweathermap.org/data/2.5"

class WeatherApp:
    """Professional Weather Application Class"""

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = BASE_URL

    def get_current_weather(self, city: str, units: str = "metric") -> Dict:
        """Fetch current weather data"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                "q": city,
                "appid": self.api_key,
                "units": units
            }
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e)}

    def get_forecast(self, city: str, units: str = "metric") -> Dict:
        """Fetch 5-day forecast data"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                "q": city,
                "appid": self.api_key,
                "units": units
            }
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e)}

    def get_air_quality(self, lat: float, lon: float) -> Dict:
        """Fetch air quality data"""
        try:
            url = "http://api.openweathermap.org/data/2.5/air_pollution"
            params = {
                "lat": lat,
                "lon": lon,
                "appid": self.api_key
            }
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e)}

    def format_current_weather(self, data: Dict, units: str) -> str:
        """Format current weather data into readable HTML"""
        if "error" in data:
            return f"<div style='color: #ef4444; padding: 20px; background: #fee2e2; border-radius: 12px;'>❌ Error: {data['error']}</div>"

        temp_unit = "°C" if units == "metric" else "°F"
        speed_unit = "m/s" if units == "metric" else "mph"

        # Extract data
        temp = data['main']['temp']
        feels_like = data['main']['feels_like']
        humidity = data['main']['humidity']
        pressure = data['main']['pressure']
        wind_speed = data['wind']['speed']
        description = data['weather'][0]['description'].title()
        icon = data['weather'][0]['icon']
        city_name = data['name']
        country = data['sys']['country']

        # Sunrise and sunset
        sunrise = datetime.fromtimestamp(data['sys']['sunrise']).strftime('%I:%M %p')
        sunset = datetime.fromtimestamp(data['sys']['sunset']).strftime('%I:%M %p')

        # Time
        current_time = datetime.now().strftime('%A, %B %d, %Y • %I:%M %p')

        html = f"""
        <div style='background: linear-gradient(135deg, #e0f2fe 0%, #ddd6fe 100%);
                    padding: 35px; border-radius: 24px;
                    box-shadow: 0 10px 40px rgba(0,0,0,0.08);
                    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;'>

            <div style='text-align: center; margin-bottom: 25px;'>
                <h2 style='margin: 0; color: #1e293b; font-size: 28px; font-weight: 600;'>
                    📍 {city_name}, {country}
                </h2>
                <p style='margin: 5px 0 0 0; color: #64748b; font-size: 14px;'>{current_time}</p>
            </div>

            <div style='text-align: center; margin-bottom: 30px;'>
                <img src='http://openweathermap.org/img/wn/{icon}@4x.png'
                     style='width: 140px; height: 140px; filter: drop-shadow(0 4px 12px rgba(0,0,0,0.1));'/>
                <h1 style='margin: 10px 0; font-size: 72px; font-weight: 700; color: #1e293b;'>
                    {temp:.1f}{temp_unit}
                </h1>
                <h3 style='margin: 5px 0; font-weight: 500; color: #475569; font-size: 20px;'>
                    {description}
                </h3>
                <p style='margin: 8px 0; color: #64748b; font-size: 15px;'>
                    Feels like {feels_like:.1f}{temp_unit}
                </p>
            </div>

            <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(140px, 1fr));
                        gap: 12px; margin-top: 25px;'>
                <div style='background: rgba(255,255,255,0.8); padding: 18px; border-radius: 16px;
                           box-shadow: 0 2px 8px rgba(0,0,0,0.04); backdrop-filter: blur(10px);'>
                    <div style='font-size: 13px; color: #64748b; margin-bottom: 6px;'>💧 Humidity</div>
                    <div style='font-size: 26px; font-weight: 700; color: #0ea5e9;'>{humidity}%</div>
                </div>
                <div style='background: rgba(255,255,255,0.8); padding: 18px; border-radius: 16px;
                           box-shadow: 0 2px 8px rgba(0,0,0,0.04); backdrop-filter: blur(10px);'>
                    <div style='font-size: 13px; color: #64748b; margin-bottom: 6px;'>🌬️ Wind Speed</div>
                    <div style='font-size: 26px; font-weight: 700; color: #06b6d4;'>{wind_speed:.1f}</div>
                    <div style='font-size: 11px; color: #94a3b8;'>{speed_unit}</div>
                </div>
                <div style='background: rgba(255,255,255,0.8); padding: 18px; border-radius: 16px;
                           box-shadow: 0 2px 8px rgba(0,0,0,0.04); backdrop-filter: blur(10px);'>
                    <div style='font-size: 13px; color: #64748b; margin-bottom: 6px;'>🌅 Sunrise</div>
                    <div style='font-size: 26px; font-weight: 700; color: #f59e0b;'>{sunrise.split()[0]}</div>
                    <div style='font-size: 11px; color: #94a3b8;'>{sunrise.split()[1]}</div>
                </div>
                <div style='background: rgba(255,255,255,0.8); padding: 18px; border-radius: 16px;
                           box-shadow: 0 2px 8px rgba(0,0,0,0.04); backdrop-filter: blur(10px);'>
                    <div style='font-size: 13px; color: #64748b; margin-bottom: 6px;'>🌇 Sunset</div>
                    <div style='font-size: 26px; font-weight: 700; color: #f97316;'>{sunset.split()[0]}</div>
                    <div style='font-size: 11px; color: #94a3b8;'>{sunset.split()[1]}</div>
                </div>
                <div style='background: rgba(255,255,255,0.8); padding: 18px; border-radius: 16px;
                           box-shadow: 0 2px 8px rgba(0,0,0,0.04); backdrop-filter: blur(10px);'>
                    <div style='font-size: 13px; color: #64748b; margin-bottom: 6px;'>🌡️ Pressure</div>
                    <div style='font-size: 26px; font-weight: 700; color: #8b5cf6;'>{pressure}</div>
                    <div style='font-size: 11px; color: #94a3b8;'>hPa</div>
                </div>
                <div style='background: rgba(255,255,255,0.8); padding: 18px; border-radius: 16px;
                           box-shadow: 0 2px 8px rgba(0,0,0,0.04); backdrop-filter: blur(10px);'>
                    <div style='font-size: 13px; color: #64748b; margin-bottom: 6px;'>👁️ Visibility</div>
                    <div style='font-size: 26px; font-weight: 700; color: #10b981;'>{data.get('visibility', 0)/1000:.1f}</div>
                    <div style='font-size: 11px; color: #94a3b8;'>km</div>
                </div>
            </div>
        </div>
        """
        return html

    def create_forecast_chart(self, data: Dict, units: str):
        """Create interactive forecast chart"""
        if "error" in data:
            return None

        temp_unit = "°C" if units == "metric" else "°F"

        # Extract forecast data
        timestamps = []
        temps = []
        feels_like = []
        humidity = []
        descriptions = []

        for item in data['list'][:16]:  # Next 48 hours
            timestamps.append(datetime.fromtimestamp(item['dt']))
            temps.append(item['main']['temp'])
            feels_like.append(item['main']['feels_like'])
            humidity.append(item['main']['humidity'])
            descriptions.append(item['weather'][0]['description'].title())

        # Create figure
        fig = go.Figure()

        # Temperature area
        fig.add_trace(go.Scatter(
            x=timestamps, y=temps,
            name='Temperature',
            fill='tozeroy',
            fillcolor='rgba(99, 179, 237, 0.2)',
            line=dict(color='#3b82f6', width=3),
            mode='lines+markers',
            marker=dict(size=8, color='#3b82f6'),
            hovertemplate='<b>%{y:.1f}' + temp_unit + '</b><br>%{text}<extra></extra>',
            text=descriptions
        ))

        # Feels like line
        fig.add_trace(go.Scatter(
            x=timestamps, y=feels_like,
            name='Feels Like',
            line=dict(color='#a78bfa', width=2, dash='dot'),
            mode='lines',
            hovertemplate='<b>%{y:.1f}' + temp_unit + '</b><extra></extra>'
        ))

        fig.update_layout(
            title={
                'text': '48-Hour Temperature Forecast',
                'font': {'size': 20, 'color': '#1e293b', 'family': 'Arial'}
            },
            xaxis_title='Date & Time',
            yaxis_title=f'Temperature ({temp_unit})',
            hovermode='x unified',
            template='plotly_white',
            height=420,
            font=dict(size=13, family='Arial'),
            plot_bgcolor='#f8fafc',
            paper_bgcolor='white',
            margin=dict(t=60, l=60, r=40, b=60),
            xaxis=dict(
                gridcolor='#e2e8f0',
                showgrid=True
            ),
            yaxis=dict(
                gridcolor='#e2e8f0',
                showgrid=True
            ),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        return fig

    def create_humidity_wind_chart(self, data: Dict, units: str):
        """Create humidity and wind speed chart"""
        if "error" in data:
            return None

        speed_unit = "m/s" if units == "metric" else "mph"

        timestamps = []
        humidity = []
        wind_speed = []

        for item in data['list'][:16]:
            timestamps.append(datetime.fromtimestamp(item['dt']))
            humidity.append(item['main']['humidity'])
            wind_speed.append(item['wind']['speed'])

        fig = go.Figure()

        # Humidity bars
        fig.add_trace(go.Bar(
            x=timestamps, y=humidity,
            name='Humidity (%)',
            marker_color='#60a5fa',
            yaxis='y',
            opacity=0.7,
            hovertemplate='<b>%{y}%</b><extra></extra>'
        ))

        # Wind speed line
        fig.add_trace(go.Scatter(
            x=timestamps, y=wind_speed,
            name=f'Wind Speed ({speed_unit})',
            line=dict(color='#34d399', width=3),
            mode='lines+markers',
            yaxis='y2',
            hovertemplate='<b>%{y:.1f} ' + speed_unit + '</b><extra></extra>'
        ))

        fig.update_layout(
            title={
                'text': 'Humidity & Wind Speed Forecast',
                'font': {'size': 20, 'color': '#1e293b', 'family': 'Arial'}
            },
            xaxis_title='Date & Time',
            yaxis=dict(
                title='Humidity (%)',
                titlefont=dict(color='#3b82f6'),
                tickfont=dict(color='#3b82f6'),
                gridcolor='#e2e8f0'
            ),
            yaxis2=dict(
                title=f'Wind Speed ({speed_unit})',
                titlefont=dict(color='#10b981'),
                tickfont=dict(color='#10b981'),
                overlaying='y',
                side='right'
            ),
            hovermode='x unified',
            template='plotly_white',
            height=380,
            plot_bgcolor='#f8fafc',
            paper_bgcolor='white',
            margin=dict(t=60, l=60, r=60, b=60),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        return fig

    def create_daily_forecast(self, data: Dict, units: str) -> str:
        """Create daily forecast summary"""
        if "error" in data:
            return "<div style='color: #ef4444; background: #fee2e2; padding: 20px; border-radius: 12px;'>Unable to fetch forecast data</div>"

        temp_unit = "°C" if units == "metric" else "°F"

        # Group by day
        daily_data = {}
        for item in data['list']:
            date = datetime.fromtimestamp(item['dt']).date()
            if date not in daily_data:
                daily_data[date] = {
                    'temps': [],
                    'descriptions': [],
                    'icons': [],
                    'humidity': [],
                    'wind': []
                }
            daily_data[date]['temps'].append(item['main']['temp'])
            daily_data[date]['descriptions'].append(item['weather'][0]['description'])
            daily_data[date]['icons'].append(item['weather'][0]['icon'])
            daily_data[date]['humidity'].append(item['main']['humidity'])
            daily_data[date]['wind'].append(item['wind']['speed'])

        # Define light gradient colors for each day (7 colors for full week)
        colors = [
            'linear-gradient(135deg, #fef3c7 0%, #fde68a 100%)',  # Light yellow
            'linear-gradient(135deg, #dbeafe 0%, #bfdbfe 100%)',  # Light blue
            'linear-gradient(135deg, #fce7f3 0%, #fbcfe8 100%)',  # Light pink
            'linear-gradient(135deg, #d1fae5 0%, #a7f3d0 100%)',  # Light green
            'linear-gradient(135deg, #e9d5ff 0%, #ddd6fe 100%)',  # Light purple
            'linear-gradient(135deg, #fed7aa 0%, #fdba74 100%)',  # Light orange
            'linear-gradient(135deg, #f3e8ff 0%, #e9d5ff 100%)'   # Light lavender
        ]

        html = "<div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(160px, 1fr)); gap: 16px;'>"

        # Show all available days (up to 7 days based on API data)
        for idx, (date, info) in enumerate(list(daily_data.items())[:7]):
            avg_temp = sum(info['temps']) / len(info['temps'])
            min_temp = min(info['temps'])
            max_temp = max(info['temps'])
            avg_humidity = sum(info['humidity']) / len(info['humidity'])
            avg_wind = sum(info['wind']) / len(info['wind'])
            description = max(set(info['descriptions']), key=info['descriptions'].count).title()
            icon = info['icons'][len(info['icons'])//2]

            day_name = date.strftime('%A')
            date_str = date.strftime('%b %d')

            html += f"""
            <div style='background: {colors[idx]};
                        padding: 22px; border-radius: 20px;
                        box-shadow: 0 4px 16px rgba(0,0,0,0.08);
                        text-align: center; transition: transform 0.2s;'>
                <div style='font-weight: 700; font-size: 17px; color: #1e293b; margin-bottom: 2px;'>
                    {day_name}
                </div>
                <div style='color: #64748b; font-size: 13px; margin-bottom: 12px;'>{date_str}</div>
                <img src='http://openweathermap.org/img/wn/{icon}@2x.png'
                     style='width: 70px; height: 70px; filter: drop-shadow(0 2px 8px rgba(0,0,0,0.1));'/>
                <div style='font-size: 32px; font-weight: 700; margin: 8px 0; color: #1e293b;'>
                    {avg_temp:.0f}{temp_unit}
                </div>
                <div style='font-size: 13px; color: #64748b; margin-bottom: 8px;'>
                    {max_temp:.0f}° / {min_temp:.0f}°
                </div>
                <div style='font-size: 12px; color: #475569; margin-bottom: 10px;
                           min-height: 36px; display: flex; align-items: center; justify-content: center;'>
                    {description}
                </div>
                <div style='display: flex; justify-content: space-around; margin-top: 10px;
                           padding-top: 10px; border-top: 1px solid rgba(0,0,0,0.1);'>
                    <div style='font-size: 11px; color: #64748b;'>
                        <div style='font-weight: 600; color: #0ea5e9;'>{avg_humidity:.0f}%</div>
                        <div>Humidity</div>
                    </div>
                    <div style='font-size: 11px; color: #64748b;'>
                        <div style='font-weight: 600; color: #10b981;'>{avg_wind:.1f}</div>
                        <div>Wind</div>
                    </div>
                </div>
            </div>
            """

        html += "</div>"
        return html

    def format_air_quality(self, data: Dict) -> str:
        """Format air quality data"""
        if "error" in data or "list" not in data:
            return "<div style='padding: 20px; background: #fef3c7; border-radius: 12px; color: #92400e;'>Air quality data unavailable for this location</div>"

        aqi = data['list'][0]['main']['aqi']
        components = data['list'][0]['components']

        # AQI levels with softer colors
        aqi_info = {
            1: ("Good", "#d1fae5", "#065f46", "Air quality is satisfactory, pose little health concern"),
            2: ("Fair", "#fef3c7", "#92400e", "Air quality is acceptable for most people"),
            3: ("Moderate", "#fed7aa", "#9a3412", "Sensitive groups should limit outdoor activity"),
            4: ("Poor", "#fecaca", "#991b1b", "Everyone may experience health effects"),
            5: ("Very Poor", "#fca5a5", "#7f1d1d", "Health alert: serious health effects for all")
        }

        level, bg_color, text_color, description = aqi_info.get(aqi, ("Unknown", "#f3f4f6", "#374151", "Data unavailable"))

        html = f"""
        <div style='background: linear-gradient(135deg, #e0f2fe 0%, #dbeafe 100%);
                    padding: 30px; border-radius: 20px;
                    box-shadow: 0 8px 32px rgba(0,0,0,0.08);'>
            <h3 style='margin: 0 0 20px 0; color: #1e293b; font-weight: 600; font-size: 22px;'>
                🌍 Air Quality Index
            </h3>
            <div style='background: {bg_color}; padding: 25px; border-radius: 16px;
                        text-align: center; margin-bottom: 20px;
                        box-shadow: 0 4px 12px rgba(0,0,0,0.06);'>
                <div style='font-size: 56px; font-weight: 700; color: {text_color}; margin-bottom: 8px;'>
                    {aqi}
                </div>
                <div style='font-size: 26px; margin: 8px 0; color: {text_color}; font-weight: 600;'>
                    {level}
                </div>
                <div style='font-size: 14px; color: {text_color}; opacity: 0.9; margin-top: 8px;'>
                    {description}
                </div>
            </div>
            <div style='background: rgba(255,255,255,0.8); padding: 20px; border-radius: 16px;
                       box-shadow: 0 2px 8px rgba(0,0,0,0.04);'>
                <div style='font-size: 15px; color: #64748b; margin-bottom: 15px; font-weight: 600;'>
                    Pollutant Levels (μg/m³)
                </div>
                <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(100px, 1fr));
                           gap: 12px; font-size: 13px;'>
                    <div style='background: #fef3c7; padding: 12px; border-radius: 10px; text-align: center;'>
                        <div style='color: #92400e; font-weight: 700; font-size: 18px;'>{components.get('pm2_5', 0):.1f}</div>
                        <div style='color: #78350f; font-size: 11px; margin-top: 4px;'>PM2.5</div>
                    </div>
                    <div style='background: #dbeafe; padding: 12px; border-radius: 10px; text-align: center;'>
                        <div style='color: #1e40af; font-weight: 700; font-size: 18px;'>{components.get('pm10', 0):.1f}</div>
                        <div style='color: #1e3a8a; font-size: 11px; margin-top: 4px;'>PM10</div>
                    </div>
                    <div style='background: #fce7f3; padding: 12px; border-radius: 10px; text-align: center;'>
                        <div style='color: #9f1239; font-weight: 700; font-size: 18px;'>{components.get('no2', 0):.1f}</div>
                        <div style='color: #831843; font-size: 11px; margin-top: 4px;'>NO₂</div>
                    </div>
                    <div style='background: #d1fae5; padding: 12px; border-radius: 10px; text-align: center;'>
                        <div style='color: #065f46; font-weight: 700; font-size: 18px;'>{components.get('o3', 0):.1f}</div>
                        <div style='color: #064e3b; font-size: 11px; margin-top: 4px;'>O₃</div>
                    </div>
                    <div style='background: #e9d5ff; padding: 12px; border-radius: 10px; text-align: center;'>
                        <div style='color: #6b21a8; font-weight: 700; font-size: 18px;'>{components.get('co', 0):.1f}</div>
                        <div style='color: #581c87; font-size: 11px; margin-top: 4px;'>CO</div>
                    </div>
                    <div style='background: #fed7aa; padding: 12px; border-radius: 10px; text-align: center;'>
                        <div style='color: #9a3412; font-weight: 700; font-size: 18px;'>{components.get('so2', 0):.1f}</div>
                        <div style='color: #7c2d12; font-size: 11px; margin-top: 4px;'>SO₂</div>
                    </div>
                </div>
            </div>
        </div>
        """
        return html

    def get_weather_alerts(self, data: Dict) -> str:
        """Format weather alerts and recommendations"""
        if "error" in data:
            return ""

        alerts = []
        temp = data['main']['temp']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']

        # Temperature alerts
        if temp > 35:
            alerts.append(("🌡️ Extreme Heat", "Temperature is very high. Stay hydrated and avoid outdoor activities.", "#fef3c7", "#92400e"))
        elif temp < 0:
            alerts.append(("❄️ Freezing Cold", "Temperature is below freezing. Dress warmly and be cautious of ice.", "#dbeafe", "#1e40af"))

        # Humidity alerts
        if humidity > 80:
            alerts.append(("💧 High Humidity", "Very humid conditions. May feel uncomfortable outdoors.", "#e0f2fe", "#0c4a6e"))
        elif humidity < 30:
            alerts.append(("🏜️ Low Humidity", "Dry conditions. Stay hydrated and use moisturizer.", "#fed7aa", "#9a3412"))

        # Wind alerts
        if wind_speed > 10:
            alerts.append(("🌬️ Strong Winds", "Windy conditions expected. Secure loose objects.", "#d1fae5", "#065f46"))

        if not alerts:
            alerts.append(("✨ Perfect Weather", "Conditions are ideal for outdoor activities!", "#d1fae5", "#065f46"))

        html = "<div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 14px; margin: 20px 0;'>"

        for title, message, bg_color, text_color in alerts:
            html += f"""
            <div style='background: {bg_color}; padding: 18px; border-radius: 14px;
                       box-shadow: 0 2px 8px rgba(0,0,0,0.06);
                       border-left: 4px solid {text_color};'>
                <div style='font-weight: 700; font-size: 15px; color: {text_color}; margin-bottom: 6px;'>
                    {title}
                </div>
                <div style='font-size: 13px; color: {text_color}; opacity: 0.9;'>
                    {message}
                </div>
            </div>
            """

        html += "</div>"
        return html

# Initialize Weather App
weather_app = WeatherApp(API_KEY)

def get_weather_data(city: str, units: str):
    """Main function to fetch and display all weather data"""
    if not city:
        return (
            "<div style='color: #ef4444; padding: 20px; background: #fee2e2; border-radius: 12px;'>⚠️ Please enter a city name</div>",
            None,
            None,
            "<div style='padding: 20px;'>No forecast data</div>",
            "",
            "<div style='padding: 20px;'>No air quality data</div>"
        )

    # Get current weather
    current = weather_app.get_current_weather(city, units)
    current_html = weather_app.format_current_weather(current, units)

    # Get weather alerts
    alerts_html = weather_app.get_weather_alerts(current) if "error" not in current else ""

    # Get forecast
    forecast = weather_app.get_forecast(city, units)
    forecast_chart = weather_app.create_forecast_chart(forecast, units)
    humidity_wind_chart = weather_app.create_humidity_wind_chart(forecast, units)
    daily_forecast = weather_app.create_daily_forecast(forecast, units)

    # Get air quality
    air_quality_html = "<div style='padding: 20px; background: #fef3c7; border-radius: 12px; color: #92400e;'>Air quality data unavailable</div>"
    if "coord" in current:
        air_quality = weather_app.get_air_quality(
            current['coord']['lat'],
            current['coord']['lon']
        )
        air_quality_html = weather_app.format_air_quality(air_quality)

    return current_html, forecast_chart, humidity_wind_chart, daily_forecast, alerts_html, air_quality_html

# Create Gradio Interface
with gr.Blocks(
    theme=gr.themes.Soft(
        primary_hue="blue",
        secondary_hue="purple",
        neutral_hue="slate"
    ),
    css="""
    .gradio-container {
        font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Roboto', sans-serif !important;
        max-width: 1400px !important;
        margin: auto !important;
    }
    .gr-button-primary {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        border: none !important;
        font-weight: 600 !important;
    }
    """
) as app:

    gr.Markdown(
        """
        <div style='text-align: center; padding: 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    border-radius: 20px; margin-bottom: 30px; box-shadow: 0 8px 32px rgba(0,0,0,0.1);'>
            <h1 style='color: white; margin: 0; font-size: 42px; font-weight: 700;'>
                🌤️ Weather Dashboard Pro
            </h1>
            <p style='color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;'>
                Real-time weather data, forecasts, and air quality information worldwide
            </p>
        </div>
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            city_input = gr.Textbox(
                label="🌍 Enter City Name",
                placeholder="e.g., London, Tokyo, New York, Chirala, Paris...",
                value="Chirala"
            )
        with gr.Column(scale=1):
            units_radio = gr.Radio(
                choices=["metric", "imperial"],
                value="metric",
                label="📏 Temperature Units",
                info="°C or °F"
            )
        with gr.Column(scale=1):
            submit_btn = gr.Button("🔍 Get Weather", variant="primary", size="lg")

    gr.Markdown("<br>")

    # Current Weather Section
    current_weather_display = gr.HTML()

    # Weather Alerts
    weather_alerts_display = gr.HTML()

    gr.Markdown("<br>")

    # Tabbed Interface for all detailed data
    with gr.Tabs() as tabs:
        with gr.Tab("📊 Temperature Forecast"):
            gr.Markdown(
                """
                <div style='text-align: center; margin: 15px 0;'>
                    <p style='color: #64748b; font-size: 14px;'>48-hour temperature trends and predictions</p>
                </div>
                """
            )
            forecast_chart_display = gr.Plot()

        with gr.Tab("💧 Humidity & Wind"):
            gr.Markdown(
                """
                <div style='text-align: center; margin: 15px 0;'>
                    <p style='color: #64748b; font-size: 14px;'>Humidity levels and wind speed forecast</p>
                </div>
                """
            )
            humidity_wind_chart_display = gr.Plot()

        with gr.Tab("📅 7-Day Forecast"):
            gr.Markdown(
                """
                <div style='text-align: center; margin: 15px 0;'>
                    <p style='color: #64748b; font-size: 14px;'>Daily weather summary for the full week</p>
                </div>
                """
            )
            daily_forecast_display = gr.HTML()

        with gr.Tab("🌍 Air Quality"):
            gr.Markdown(
                """
                <div style='text-align: center; margin: 15px 0;'>
                    <p style='color: #64748b; font-size: 14px;'>Air quality index and pollutant levels</p>
                </div>
                """
            )
            air_quality_display = gr.HTML()

    gr.Markdown(
        """
        <div style='text-align: center; padding: 30px; background: #f8fafc; border-radius: 16px;
                    margin-top: 40px; border: 1px solid #e2e8f0;'>
            <p style='color: #64748b; margin: 0; font-size: 14px;'>
                ⚡ Data provided by OpenWeatherMap API • Updated in real-time
            </p>
            <p style='color: #94a3b8; margin: 10px 0 0 0; font-size: 13px;'>
                💡 Try cities worldwide: London, Paris, Tokyo, New York, Mumbai, Sydney, Dubai, Toronto
            </p>
        </div>
        """
    )

    # Button click event
    submit_btn.click(
        fn=get_weather_data,
        inputs=[city_input, units_radio],
        outputs=[
            current_weather_display,
            forecast_chart_display,
            humidity_wind_chart_display,
            daily_forecast_display,
            weather_alerts_display,
            air_quality_display
        ]
    )

    # Also trigger on Enter key
    city_input.submit(
        fn=get_weather_data,
        inputs=[city_input, units_radio],
        outputs=[
            current_weather_display,
            forecast_chart_display,
            humidity_wind_chart_display,
            daily_forecast_display,
            weather_alerts_display,
            air_quality_display
        ]
    )

# Launch the app
print("=" * 60)
print("🚀 WEATHER DASHBOARD PRO - LAUNCHING...")
print("=" * 60)
print("\n📍 Your API key is configured and ready!")
print("\n✨ FEATURES INCLUDED:")
print("   • Current weather with beautiful light-colored cards")
print("   • Smart weather alerts and recommendations")
print("   • 48-hour temperature forecast chart")
print("   • Humidity & wind speed visualization")
print("   • 7-day full week forecast with gradient cards")
print("   • Air quality index with pollutant levels")
print("   • Support for Celsius and Fahrenheit")
print("   • Smooth animations and modern UI")
print("\n🎨 DESIGN ENHANCEMENTS:")
print("   • Light pastel color schemes")
print("   • Soft shadows and smooth transitions")
print("   • Professional typography")
print("   • Responsive grid layouts")
print("\n⏳ Please wait while the interface loads...")
print("=" * 60)
print()

app.launch(share=True, debug=True)

/tmp/ipython-input-3647669742.py:576: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-3647669742.py:576: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(


🚀 WEATHER DASHBOARD PRO - LAUNCHING...

📍 Your API key is configured and ready!

✨ FEATURES INCLUDED:
   • Current weather with beautiful light-colored cards
   • Smart weather alerts and recommendations
   • 48-hour temperature forecast chart
   • Humidity & wind speed visualization
   • 7-day full week forecast with gradient cards
   • Air quality index with pollutant levels
   • Support for Celsius and Fahrenheit
   • Smooth animations and modern UI

🎨 DESIGN ENHANCEMENTS:
   • Light pastel color schemes
   • Soft shadows and smooth transitions
   • Professional typography
   • Responsive grid layouts

⏳ Please wait while the interface loads...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://45bbff11af06e00ba4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory